In [ ]:
from admcycles import *

In [62]:
def make_glu_graph(g, i, n=1):
    """
    Constructs the StableGraph Gi with i elliptic tails efficiently.
    """
    if i == 0:
        return StableGraph([g], [list(range(1, n + 1))], [])
    
    # V: Main vertex (genus g-i) + i tails (genus 1)
    V = [g - i] + [1] * i
    # H: Main legs (1..n + internal) + Tail legs
    H = [list(range(1, n + i + 1))] + [[n + i + k] for k in range(1, i + 1)]
    # E: Edges connecting Main(n+k) -- Tail(n+i+k)
    E = [(n + k, n + i + k) for k in range(1, i + 1)]
    
    return StableGraph(V, H, E)

In [63]:
from itertools import product

def pslambdaclass_term(d, g, n, i):
    """
    Returns the i-th term of the pslambdaclass expansion efficiently.
    """
    if i == 0:
        return lambdaclass(d, g, n)
        
    G = make_glu_graph(g, i, n)
    
    # Main decoration: Lambda class on vertex 0
    # Tails (indices 1 to i): Identity (fundclass)
    decorations = [lambdaclass(d - i, g - i, n + i)] + [fundclass(1, 1)] * i
    
    norm = 1 / factorial(i)
    return norm * G.boundary_pushforward(decorations)

In [64]:
from itertools import product

def OtherSide_term(g, n, i):
    """
    Returns ONLY the i-th term of the OtherSide sum.
    Expansion: (-1)^i * Sum_{bits} (-1)^sum(bits) * G * (psi_prod)
    This corresponds to prod(psi_dot - psi_star) on the edges.
    """
    if i == 0:
        return fundclass(g, n)

    G = make_glu_graph(g, i, n)
    norm = 1 / factorial(i)
    
    # We want to push forward prod(psi_dot - psi_star) on all i edges.
    # We can reuse the logic from boundary_pushforward_optimized!
    
    # Define edge behaviors for OtherSide:
    # Edge k: contributes (psi_dot - psi_star)
    # Note: psi_dot is on the tail (vertex k), psi_star is on main (vertex 0)
    
    # We will build this manually using the efficient prodtautclass method
    # to avoid creating 2^i graph pushforwards.
    
    edge_indices = list(range(i)) # All edges 0 to i-1
    
    # Pre-compute edge data
    edge_data = []
    all_edges = G.edges()
    
    for idx in edge_indices:
        # For Gi, we know the structure:
        # Edge idx connects Vertex 0 (main) and Vertex idx+1 (tail)
        # psi_star is on V0, leg n+1+idx
        # psi_dot is on V(idx+1), leg n+i+1+idx
        
        # PSI STAR (on main vertex 0)
        l_star = n + 1 + idx
        idx_star = G.legs(0).index(l_star) + 1
        psi_star = psiclass(idx_star, g - i, n + i) # Positive psi
        
        # PSI DOT (on tail vertex idx+1)
        l_dot = n + i + 1 + idx
        idx_dot = 1 # Tail only has 1 leg
        psi_dot = psiclass(idx_dot, 1, 1) # Positive psi
        
        edge_data.append({'psi_star': psi_star, 'psi_dot': psi_dot})

    all_terms = []
    
    # Iterate 2^i terms for the expansion of prod(psi_dot - psi_star)
    for bits in product([0, 1], repeat=i):
        # bits: 0->psi_dot, 1->(-psi_star)
        
        # Start with Identity
        current_term_classes = [fundclass(G.genera(v), G.num_legs(v)) for v in range(G.num_verts())]
        
        sign = 1
        
        for k, bit in enumerate(bits):
            if bit == 0: # psi_dot
                v_tail = k + 1
                current_term_classes[v_tail] = current_term_classes[v_tail] * edge_data[k]['psi_dot']
            else: # -psi_star
                v_main = 0
                current_term_classes[v_main] = current_term_classes[v_main] * edge_data[k]['psi_star']
                sign *= -1

        # Check for zero classes
        if any(len(c._terms) == 0 for c in current_term_classes):
            continue
            
        # Unwrap to decstratum
        decs = [list(c._terms.values())[0] for c in current_term_classes]
        
        # Apply the global sign and coeff for this term
        # (We multiply the first element of the list by the scalar)
        if sign == -1:
            decs[0].poly.coeff = [-c for c in decs[0].poly.coeff]
            
        all_terms.append(decs)

    if not all_terms:
        return 0

    # Global Factor: (-1)^i * (1/i!)
    global_factor = ((-1)**i) * norm
    
    result = prodtautclass(G, all_terms).pushforward()
    return global_factor * result

In [49]:
from itertools import product

def boundary_pushforward_optimized(G, edge_indices):
    """
    Robust optimized version.
    Manually expands terms, filters out zero classes, converts to decstratum,
    and pushes forward in one go.
    """
    
    # --- HELPER: Unwrap TautologicalClass to decstratum ---
    def to_dec(taut_class):
        # We assume the class is non-zero (filtered before calling this)
        return list(taut_class._terms.values())[0]

    # --- 1. Pre-computation ---
    edge_data = []
    all_edges = G.edges()

    for edge_idx in edge_indices:
        edge = all_edges[edge_idx] 
        l1, l2 = edge[0], edge[1]
        
        # Find Vertex 1
        v1 = next(v for v in range(G.num_verts()) if l1 in G.legs(v))
        idx1 = G.legs(v1).index(l1) + 1
        psi1 = -psiclass(idx1, G.genera(v1), G.num_legs(v1))
        
        # Find Vertex 2
        v2 = next(v for v in range(G.num_verts()) if l2 in G.legs(v))
        idx2 = G.legs(v2).index(l2) + 1
        psi2 = -psiclass(idx2, G.genera(v2), G.num_legs(v2))
        
        edge_data.append({
            'v1': v1, 'psi1': psi1,
            'v2': v2, 'psi2': psi2
        })

    # --- 2. Build terms ---
    all_terms = []

    for bits in product([0, 1], repeat=len(edge_indices)):
        
        # Start with Identity (fundclass) on all vertices
        current_term_classes = [fundclass(G.genera(v), G.num_legs(v)) for v in range(G.num_verts())]
        
        # Multiply in the psi classes
        for i, bit in enumerate(bits):
            data = edge_data[i]
            if bit == 0:
                v = data['v1']
                current_term_classes[v] = current_term_classes[v] * data['psi1']
            else:
                v = data['v2']
                current_term_classes[v] = current_term_classes[v] * data['psi2']
        
        # --- CRITICAL FIX: Filter out zero terms ---
        # If any vertex class vanished (e.g. psi^2 on dim 1), the whole term is 0.
        # In admcycles, a zero class has an empty _terms dictionary.
        if any(len(c._terms) == 0 for c in current_term_classes):
            continue 

        # CONVERT to decstratums
        current_term_decs = [to_dec(c) for c in current_term_classes]
        all_terms.append(current_term_decs)

    # --- 3. Create & Pushforward ---
    # If all terms vanished, return 0
    if not all_terms:
        return TautologicalClass(G.to_mean_graph().parent(), {})

    big_class = prodtautclass(G, all_terms)
    return big_class.pushforward()

In [66]:
g = 4
n = 1

# Create an array of Graphs G where G[i] has i elliptic tails
# ranges from 0 to g
G = [make_glu_graph(g, i, n) for i in range(g + 1)]

# Create an array of Tautological Classes GLU where GLU[i] corresponds to G[i]
GLU = [graph.to_tautological_class() for graph in G]


# --- 3. The Verification ---

# Now you can just use indices! 
# Example: G[3] is the graph with 3 edges.

# Calculate the bicolored terms
# G[3] has 3 edges (0, 1, 2). We want the first two: [0, 1]
G2Bic = boundary_pushforward_optimized(G[2], [0,1])
G3Bic = boundary_pushforward_optimized(G[3], [0])

# G[4] has 4 edges. We want the first one: [0]
#G4Bic = boundary_pushforward_optimized(G[4], [0])

# Verify the relation: GLU2 * GLU3 - ...
#result = (GLU[2] * GLU[3]) - (6 * G3Bic + 6 * G4Bic + GLU[5])

result = (GLU[2] * GLU[2]) - (2*boundary_pushforward_optimized(G[2],[0,1])+4*boundary_pushforward_optimized(G[3],[0])+GLU[4])

print("Is the relation zero?", result.is_zero())

Is the relation zero? True


In [68]:
from itertools import product # --- 3. The Generalized Multiplier ---

def multiply_glu_terms(g, i, j, k, n=1):
    """
    Computes the contribution to the product of:
      Blue Term: pslambdaclass_term(d=g, i)
      Red Term:  OtherSide_term(j)
    Intersecting along 'k' edges.
    """
    
    # --- GUARD CLAUSE: Impossible Intersections ---
    # We cannot overlap more edges than exist in the source graphs.
    if k > i or k > j or k < 0:
        return 0
    
    # 1. Graph Geometry
    # The resulting graph has (i + j - k) edges
    total_edges = i + j - k
    G = make_glu_graph(g, total_edges, n)
    
    # 2. Edges Logic
    # We map the indices 0..total_edges-1 to their roles:
    # 0 to k-1:          BICOLORED (Overlap)
    # k to j-1:          RED ONLY  (From OtherSide)
    # j to total_edges-1: BLUE ONLY (From pslambda)
    
    # 3. Lambda Distribution
    # Main vertex degree check
    deg_main = (g - i) - (j - k)
    if deg_main < 0:
        return 0 
        
    lambda_main = lambdaclass(deg_main, g - total_edges, n + total_edges)
    lambda_tail = lambdaclass(1, 1, 1) 
    
    # 4. Build Edge Data
    edge_data = []
    
    # We need data for ALL edges in the resulting graph (0 to total_edges-1)
    for idx in range(total_edges):
        l_star = n + 1 + idx
        idx_star = G.legs(0).index(l_star) + 1
        psi_star = psiclass(idx_star, g - total_edges, n + total_edges)
        
        # Tail leg is always index 1
        psi_dot = psiclass(1, 1, 1)
        
        edge_data.append({'psi_star': psi_star, 'psi_dot': psi_dot})

    # 5. Expand Polynomials
    # We loop over the 'j' edges coming from the Red Graph (OtherSide)
    # These cover indices 0 to j-1.
    all_terms = []
    
    for bits in product([0, 1], repeat=j):
        current_classes = [fundclass(1,1) for _ in range(G.num_verts())]
        current_classes[0] = lambda_main
        
        # Apply Lambda_1 to Red-Only Tails (indices k to j-1)
        for red_idx in range(k, j):
             current_classes[red_idx + 1] = lambda_tail
             
        sign = 1
        valid_term = True

        for bit_idx, bit in enumerate(bits):
            edge_idx = bit_idx 
            data = edge_data[edge_idx]
            
            if edge_idx < k:
                # --- BICOLORED (0 to k-1) ---
                if bit == 0: 
                    current_classes[edge_idx + 1] *= (data['psi_dot'] * data['psi_dot'])
                else: 
                    current_classes[0] *= (data['psi_star'] * data['psi_star'])
                    sign *= -1
            else:
                # --- RED ONLY (k to j-1) ---
                if bit == 0:
                    current_classes[edge_idx + 1] *= data['psi_dot']
                else:
                    current_classes[0] *= data['psi_star']
                    sign *= -1

        if any(len(c._terms) == 0 for c in current_classes):
            continue

        decs = [list(c._terms.values())[0] for c in current_classes]
        
        if sign == -1:
             decs[0].poly.coeff = [-c for c in decs[0].poly.coeff]
             
        all_terms.append(decs)

    if not all_terms:
        return 0
    
    return prodtautclass(G, all_terms).pushforward()

In [ ]:
g = 4
n = 1
i = 2
j = 2

# Terms for k = 2, 1, 0
# Coefficients are (i choose k) * (j choose k) * k!
# For i=2, j=2:
# k=2: (1)*(1)*2 = 2
# k=1: (2)*(2)*1 = 4
# k=0: (1)*(1)*1 = 1

term_k2 = 2 * multiply_glu_terms(g, i, j, 2, n) # Overlap 2 -> Result G[2]
term_k1 = 4 * multiply_glu_terms(g, i, j, 1, n) # Overlap 1 -> Result G[3]
term_k0 = 1 * multiply_glu_terms(g, i, j, 0, n) # Overlap 0 -> Result G[4]

RHS = term_k2 + term_k1 + term_k0
LHS = pslambdaclass_term(g, g, n, i) * OtherSide_term(g, n, j)

result = LHS - RHS
print("Is the relation zero?", result.is_zero())